## Imports

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
import warnings
warnings.filterwarnings(action='once')
data_path = str(Path(os.getcwd()).parent.absolute())+"/data"
figures_path = str(Path(os.getcwd()).parent.absolute())+"/reports/figures"

# Load Data

In [ ]:
factors = pd.read_csv(data_path+"/interim/factors.csv", index_col=0).loc['2007-02-28':,:]
portfolio_excess_returns = pd.read_csv(data_path+"/interim/portfolio_excess_returns.csv", index_col=0).loc['2007-02-28':'2017-10-31',:]

In [ ]:
green_factors = pd.read_csv("green_factors.csv", index_col=0).loc['2007-02-28':'2017-10-31',:]
green_factors = green_factors.loc[:,['ESG_ADJ', 'E_V_IND_W', 'S_V_IND_W', 'G_V_IND_W', 'CO2_V_IND_W']]

## Function
Based on Feng, Giglio, and Xiu (2020) matlab code

In [ ]:
def infer(Ri, gt, ht, sel1, sel2, sel3):
    n = Ri.shape[1]
    p = ht.shape[1]
    d = gt.shape[1]
    l = gt.shape[0]

    cov_h = np.cov(ht.T,Ri.T)[len(ht.columns):,:len(ht.columns)]
    cov_g = np.cov(gt.T,Ri.T)[len(gt.columns):,:len(gt.columns)]
    ER  = pd.DataFrame({'Average Excess Returns': Ri.mean()})
    ones_n = np.ones((n, 1))
    M0 = np.eye(n) - ones_n @ np.linalg.inv(ones_n.T @ ones_n) @ ones_n.T
    select = np.unique(np.concatenate([sel1, sel2]))
    select = [ht.columns.get_loc(c) for c in select if c in ht]
    X = np.hstack([cov_g, cov_h[:,select]])
    lambda_full = np.linalg.inv(np.dot(np.dot(X.T, M0), X)).dot(np.dot(np.dot(X.T, M0), ER))
    lambdag = lambda_full[:d]
    zthat = np.full((d, l), np.nan)
    select3 = [ht.columns.get_loc(c) for c in sel3 if c in ht]
    ht2=np.array(ht.T)
    for i in range(d):
        M_mdl = np.eye(l) - np.dot(np.dot(ht.iloc[:,select3], np.linalg.inv(np.dot(ht.iloc[:,select3].T, ht.iloc[:,select3]))), ht.iloc[:,select3].T)
        zthat[i, :] = np.dot(M_mdl, gt.iloc[:,i])
    Sigmazhat = np.dot(zthat,zthat.T )/ l
    temp2 = 0
    ii = 0
    for row in range(0,l):
        ii += 1
        mt = 1 - np.dot(lambda_full.T , np.hstack( [gt.iloc[row,:].T , ht.iloc[row,select].T  ]))
        temp2 += mt**2 * np.dot(np.dot(np.linalg.inv(Sigmazhat), zthat[:, ii-1][:, np.newaxis]), np.dot(zthat[:, ii-1][:, np.newaxis].T, np.linalg.inv(Sigmazhat)))

    avar_lambdag = np.diag(temp2) / l
    se = np.sqrt(avar_lambdag / l)

    vt = np.vstack([gt.T, ht.iloc[:,select].T])
    V_bar = vt - np.mean(vt, axis=1, keepdims=True) * np.ones((1, l))
    var_v = np.dot(V_bar , V_bar.T) / l
    gamma = np.diag(var_v) * lambda_full.T

    tstat = (lambdag / se)[0,0]
    lambda_bp = gamma[0,0]*10000

    return [lambda_bp, tstat]

## Computing and saving the results

In [ ]:
Ri = portfolio_excess_returns
ht = factors
sel1 = list(pd.read_csv(data_path + '/processed/sel1_results.csv',index_col=0).iloc[0,1:])
selFF = ['SMB','HML','MktRF']
main = pd.DataFrame(columns=['TestList', 'Characterisc Name', 'lambda_ds', 'tstat_ds', 'lambda_ss', 'tstat_ss'])
for green_characteristic in green_factors.columns:
    gt = pd.DataFrame(data = green_factors.loc[:, green_characteristic])
    sel2 = list(pd.read_csv(data_path + '/processed/'+green_characteristic+'_sel2_results.csv',index_col=0).iloc[0,1:])
    sel3 = list(pd.read_csv(data_path + '/processed/'+green_characteristic+'_sel3_results.csv',index_col=0).iloc[0,1:])

    print([len(ht.columns),green_characteristic] + infer(Ri, gt, ht, sel1, sel2, sel3) + infer(Ri, gt, ht, sel1, [], sel3) + infer(Ri, gt, ht, selFF, [], sel3))
    main.loc[len(main),['TestList', 'Characterisc Name', 'lambda_ds', 'tstat_ds', 'lambda_ss', 'tstat_ss', 'lambda_FF', 'tstat_FF']] = [len(ht.columns),green_characteristic] + infer(Ri, gt, ht, sel1, sel2, sel3) + infer(Ri, gt, ht, sel1, [], sel3)  + infer(Ri, gt, ht, selFF, [], sel3)
main.to_csv(data_path + '/processed/main.csv')
